https://www.kaggle.com/cdeotte/200-magical-models-santander-0-920

In [38]:
# LOAD LIBRARIES
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd, numpy as np, gc
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import statsmodels.api as sm
import tqdm

In [39]:
# GET INDICIES OF REAL TEST DATA FOR FE
#######################
# TAKE FROM YAG320'S KERNEL
# https://www.kaggle.com/yag320/list-of-fake-samples-and-public-private-lb-split

test_path = '../input/test.csv'

df_test = pd.read_csv(test_path)
df_test.drop(['ID_code'], axis=1, inplace=True)
df_test = df_test.values

unique_samples = []
unique_count = np.zeros_like(df_test)
for feature in range(df_test.shape[1]):
    _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
    unique_count[index_[count_ == 1], feature] += 1

# Samples which have unique values are real the others are fake
real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

print('Found',len(real_samples_indexes),'real test')
print('Found',len(synthetic_samples_indexes),'fake test')

###################

Found 100000 real test
Found 100000 fake test


In [3]:
d = {}
for i in range(200): d['var_'+str(i)] = 'float32'
d['target'] = 'uint8'
d['ID_code'] = 'object'

train = pd.read_csv('../input/train.csv', dtype=d)
test = pd.read_csv('../input/test.csv', dtype=d)

print('Loaded',len(train),'rows of train')
print('Loaded',len(test),'rows of test')

Loaded 200000 rows of train
Loaded 200000 rows of test


In [8]:
d

{'var_0': 'float32',
 'var_1': 'float32',
 'var_2': 'float32',
 'var_3': 'float32',
 'var_4': 'float32',
 'var_5': 'float32',
 'var_6': 'float32',
 'var_7': 'float32',
 'var_8': 'float32',
 'var_9': 'float32',
 'var_10': 'float32',
 'var_11': 'float32',
 'var_12': 'float32',
 'var_13': 'float32',
 'var_14': 'float32',
 'var_15': 'float32',
 'var_16': 'float32',
 'var_17': 'float32',
 'var_18': 'float32',
 'var_19': 'float32',
 'var_20': 'float32',
 'var_21': 'float32',
 'var_22': 'float32',
 'var_23': 'float32',
 'var_24': 'float32',
 'var_25': 'float32',
 'var_26': 'float32',
 'var_27': 'float32',
 'var_28': 'float32',
 'var_29': 'float32',
 'var_30': 'float32',
 'var_31': 'float32',
 'var_32': 'float32',
 'var_33': 'float32',
 'var_34': 'float32',
 'var_35': 'float32',
 'var_36': 'float32',
 'var_37': 'float32',
 'var_38': 'float32',
 'var_39': 'float32',
 'var_40': 'float32',
 'var_41': 'float32',
 'var_42': 'float32',
 'var_43': 'float32',
 'var_44': 'float32',
 'var_45': 'float32'

In [9]:
#value_counts()
train['var_0'].value_counts()

10.682900    11
13.065600    11
8.664900     11
11.697600    10
10.678300    10
11.959000    10
8.942500     10
9.588200     10
11.783700    10
9.837900     10
10.946800    10
8.726000     10
9.511400     10
11.291600    10
10.736900    10
8.912900     10
12.927100    10
11.302100     9
12.887800     9
12.760800     9
9.788800      9
12.423900     9
7.490800      9
8.451700      9
8.185400      9
12.719800     9
11.374400     9
11.340200     9
8.666400      9
8.337600      9
             ..
14.891100     1
12.758300     1
10.621200     1
14.863700     1
15.103100     1
14.863800     1
9.420400      1
11.046000     1
7.981000      1
18.588301     1
8.533800      1
14.230100     1
10.347900     1
8.800700      1
6.044600      1
14.511900     1
12.765200     1
5.533200      1
13.273000     1
5.225000      1
14.166700     1
6.229700      1
6.646800      1
9.420000      1
18.586901     1
13.293400     1
5.453100      1
18.586700     1
5.341800      1
6.145400      1
Name: var_0, Length: 946

In [10]:
# FREQUENCY ENCODE
def encode_FE(df, col, test):
    cv = df[col].value_counts()
    nm = col + '_FE'
    df[nm] = df[col].map(cv)
    test[nm] = test[col].map(cv)
    test[nm].fillna(0, inplace=True)
    if cv.max()<=255:
        df[nm] = df[nm].astype('uint8')
        test[nm] = test[nm].astype('uint8')
    else:
        df[nm] = df[nm].astype('uint16')
        test[nm] = test[nm].astype('uint16')        
    return

In [13]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.522699,-2.3978,7.8784,8.5635,12.780300,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.430500,2.0339,8.1267,8.7889,18.356001,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.604200,3.1417,-6.5213,8.2675,14.722200,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.034700,-1.2706,-2.9275,10.2922,17.969700,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.287600,-1.5121,3.9267,9.5031,17.997400,-8.8104


In [14]:
test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.267500,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.711200,2.4669,4.3654,10.7200,15.472200,-8.719700
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.631599,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.576500,0.4773,-1.4852,9.8714,19.129299,-20.976000
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.253700,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.981300,2.1281,-7.1086,7.0618,19.895599,-23.179399
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.566000,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.187400,3.1656,3.9567,9.2295,13.016800,-4.210800
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.604800,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.554199,-0.2860,-5.1612,7.2882,13.926000,-9.184600


In [15]:
test['target'] = -1
comb = pd.concat([train,test.loc[real_samples_indexes]],axis=0,sort=True) #combine training data and real samples test data -> 300,000 rows
for i in range(200):
    encode_FE(comb,'var_'+str(i),test)

In [16]:
comb.head()

,ID_code,target,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,...,var_190_FE,var_191_FE,var_192_FE,var_193_FE,var_194_FE,var_195_FE,var_196_FE,var_197_FE,var_198_FE,var_199_FE
0,train_0,0,8.9255,-6.7863,2.9252,9.476300,13.310200,26.5376,1.4403,14.7100,...,3,8,9,4,9,5,5,14,5,2
1,train_1,0,11.5006,-4.1473,-0.4032,-13.695000,8.406800,35.4734,1.7093,15.1866,...,8,5,9,2,4,4,4,21,6,2
2,train_2,0,8.6093,-2.7457,-0.3249,-0.393900,12.631700,14.8863,1.3854,15.0284,...,4,5,6,2,2,2,3,12,4,2
3,train_3,0,11.0604,-2.1518,2.3061,-19.859200,22.531601,18.6129,1.3512,9.3291,...,2,3,6,4,4,8,5,4,2,2
4,train_4,0,9.8369,-1.4834,-9.4458,-22.926399,12.356200,17.3410,1.6940,7.1179,...,3,8,1,1,1,9,5,9,2,2


In [17]:
comb.tail()

,ID_code,target,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,...,var_190_FE,var_191_FE,var_192_FE,var_193_FE,var_194_FE,var_195_FE,var_196_FE,var_197_FE,var_198_FE,var_199_FE
199986,test_199986,-1,19.288401,-2.8384,3.1869,-14.2318,6.170900,27.691601,1.5836,10.6221,...,6,3,9,4,3,8,5,20,3,3
199993,test_199993,-1,14.676400,-8.1066,-3.9173,7.8704,7.879700,25.814600,1.5433,10.9763,...,3,5,6,3,8,6,3,7,6,1
199995,test_199995,-1,13.167800,1.0136,2.6802,0.0951,9.751700,28.611900,1.7091,13.6924,...,1,6,3,2,1,7,2,15,3,1
199996,test_199996,-1,9.717100,-9.1462,3.2618,-6.1583,20.444099,21.842100,1.7256,8.5803,...,1,6,10,3,2,11,3,15,6,1
199999,test_199999,-1,10.466400,1.8070,8.4796,0.3854,12.125000,27.860201,1.2240,9.9291,...,4,6,5,4,4,8,3,6,6,1


In [20]:
train = comb[:len(train)]; del comb
print('Added 200 new magic features!')

Added 200 new magic features!


In [21]:
train.head()

,ID_code,target,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,...,var_190_FE,var_191_FE,var_192_FE,var_193_FE,var_194_FE,var_195_FE,var_196_FE,var_197_FE,var_198_FE,var_199_FE
0,train_0,0,8.9255,-6.7863,2.9252,9.476300,13.310200,26.5376,1.4403,14.7100,...,3,8,9,4,9,5,5,14,5,2
1,train_1,0,11.5006,-4.1473,-0.4032,-13.695000,8.406800,35.4734,1.7093,15.1866,...,8,5,9,2,4,4,4,21,6,2
2,train_2,0,8.6093,-2.7457,-0.3249,-0.393900,12.631700,14.8863,1.3854,15.0284,...,4,5,6,2,2,2,3,12,4,2
3,train_3,0,11.0604,-2.1518,2.3061,-19.859200,22.531601,18.6129,1.3512,9.3291,...,2,3,6,4,4,8,5,4,2,2
4,train_4,0,9.8369,-1.4834,-9.4458,-22.926399,12.356200,17.3410,1.6940,7.1179,...,3,8,1,1,1,9,5,9,2,2


In [22]:
test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190_FE,var_191_FE,var_192_FE,var_193_FE,var_194_FE,var_195_FE,var_196_FE,var_197_FE,var_198_FE,var_199_FE
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.267500,2.1337,...,2,2,2,7,2,3,5,4,4,4
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.631599,-4.4131,...,2,6,8,1,4,18,2,11,3,3
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.253700,1.5233,...,2,3,9,2,4,2,1,3,2,1
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.566000,3.3755,...,3,2,7,3,4,4,2,10,7,1
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.604800,2.9890,...,3,4,8,4,5,11,2,4,2,3


In [23]:
#LGBM param
param = {
    'learning_rate': 0.04,
    'num_leaves': 3,
    'metric':'auc',
    'boost_from_average':'false',
    'feature_fraction': 1.0,
    'max_depth': -1,
    'objective': 'binary',
    'verbosity': -10}

In [24]:
from sklearn.model_selection import StratifiedKFold, KFold

In [29]:
num_folds = 5
features = [c for c in train.columns if c not in ['ID_code', 'target']]
target = train['target']

In [30]:
folds = StratifiedKFold(n_splits=num_folds, random_state=40000)
oof = np.zeros(len(train)) #oof: out of fold
getVal = np.zeros(len(train)) #no need?
predictions = np.zeros(len(target))
feature_importance_df = pd.DataFrame()

In [35]:
print('oof size: {}'.format(oof.shape))
print('getVal size: {}'.format(getVal.shape))
print('predictions size: {}'.format(predictions.shape))
print('feature_importance_df size: {}'.format(feature_importance_df.shape))

oof size: (200000,)
getVal size: (200000,)
predictions size: (200000,)
feature_importance_df size: (0, 0)


In [36]:
folds.split(train.values, target.values)

<generator object _BaseKFold.split at 0x10adfcf68>

In [43]:
print('Light GBM Model')
for fold_, (trn_index, val_index) in enumerate(folds.split(train.values, target.values)):
    X_train, y_train = train.iloc[trn_index][features], target.iloc[trn_index] #prepare training set
    X_val, y_val = train.iloc[val_index][features], target.iloc[val_index] #prepare validation set
    
    print("Fold idx:{}".format(fold_ + 1))
    #create data set in the fold
    trn_data = lgb.Dataset(X_train, label=y_train) #training set
    val_data = lgb.Dataset(X_val, label=y_val) #validation set
    
    #create and train model
    clf = lgb.train(param, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 3000)
    #valid_setsにtrn_dataとval_dataのリストを渡すことで、それぞれのaucがされる
    
    #predict oof
    oof[val_index] = clf.predict(X_val, num_iteration=clf.best_iteration) #最もよかったモデルで予測
    
    #predict test (has to be divided by the number of total folds in order to make average)
    predictions += clf.predict(test[features],num_iteration=clf.best_iteration) / folds.n_splits
    
    #summarize feature importance in the fold
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    
    #add feature importance in the fold
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

Light GBM Model
Fold idx:1
Training until validation scores don't improve for 3000 rounds.
[5000]	training's auc: 0.931329	valid_1's auc: 0.907737
[10000]	training's auc: 0.94722	valid_1's auc: 0.911946
[15000]	training's auc: 0.958133	valid_1's auc: 0.912275
Early stopping, best iteration is:
[14508]	training's auc: 0.957167	valid_1's auc: 0.912441
Fold idx:2
Training until validation scores don't improve for 3000 rounds.
[5000]	training's auc: 0.93195	valid_1's auc: 0.905981
[10000]	training's auc: 0.947779	valid_1's auc: 0.910135
Early stopping, best iteration is:
[11369]	training's auc: 0.951047	valid_1's auc: 0.910427
Fold idx:3
Training until validation scores don't improve for 3000 rounds.
[5000]	training's auc: 0.931034	valid_1's auc: 0.910117
[10000]	training's auc: 0.94707	valid_1's auc: 0.913901
[15000]	training's auc: 0.958105	valid_1's auc: 0.913794
Early stopping, best iteration is:
[12175]	training's auc: 0.952256	valid_1's auc: 0.914031
Fold idx:4
Training until validat

In [44]:
#check CV score
print("\n >> CV score: {:<8.5f}".format(roc_auc_score(target, oof)))


 >> CV score: 0.91307 


In [46]:
#Submission
submission = pd.DataFrame({"ID_code": test.ID_code.values})
submission["target"] = predictions
submission.to_csv("submission.csv", index=False)